In [ ]:

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.preprocessing import LabelEncoder

# 1. Load the dataset
data = pd.read_csv('heart_disease.csv')  # Update with your path if needed

# 2. Encode categorical variables
label_encoders = {}
for column in data.columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

print("? Encoded Heart Disease Data Sample:")
print(data.head())

# 3. Define the Bayesian Network structure
model = DiscreteBayesianNetwork([
    ('sex', 'cp'),
    ('cp', 'target'),
    ('fbs', 'target'),
    ('exang', 'target')
])

# 4. Train using Maximum Likelihood Estimation
model.fit(data, estimator=MaximumLikelihoodEstimator)

# 5. Set up inference
infer = VariableElimination(model)

# Show available values for input
print("\n?? Available class labels:")
for col, le in label_encoders.items():
    print(f"{col}: {list(le.classes_)}")

# 6. Take input from user
print("\n?? Enter values for prediction:")
cp_input = input("Chest Pain Type (e.g., 'asymptomatic'): ")
fbs_input = input("Fasting Blood Sugar > 120 mg/dl? (true/false): ")

# Encode input
try:
    cp_val = label_encoders['cp'].transform([cp_input])[0]
    fbs_val = label_encoders['fbs'].transform([fbs_input])[0]
except ValueError as e:
    print(f"? Invalid input: {e}")
    exit()

# 7. Make prediction
result = infer.query(variables=['target'], evidence={'cp': cp_val, 'fbs': fbs_val})
target_labels = label_encoders['target'].inverse_transform(range(len(result.values)))

print(f"\n?? Prediction Result for input (cp='{cp_input}', fbs='{fbs_input}'):")
for label, prob in zip(target_labels, result.values):
    print(f"  ? Heart Disease = {label}: Probability = {prob:.4f}")

# 8. Visualize the Bayesian Network
plt.figure(figsize=(8, 5))
G = nx.DiGraph()
G.add_edges_from(model.edges())
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color='lightcoral',
        font_size=12, font_weight='bold', edge_color='black')
plt.title("?? Heart Disease Bayesian Network")
plt.show()
